In [94]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, col

In [3]:
spark = SparkSession.builder.appName('notebook_session_2').getOrCreate()

22/10/24 16:09:21 WARN Utils: Your hostname, mfedorchuk-pro15.local resolves to a loopback address: 127.0.0.1; using 172.16.66.36 instead (on interface en0)
22/10/24 16:09:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/24 16:09:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/24 16:09:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
actor_df = spark.read.csv('./data/actor.csv', headder=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)


1. Вивести кількість фільмів в кожній категорії. Результат відсортувати за спаданням.

In [79]:
category_film_set = film_df.join(film_category_df, on=film_df.film_id == film_category_df.film_id, how="left").join(
    category_df, on=film_category_df.category_id == category_df.category_id, how="left").select(
    film_df.film_id, category_df.name).groupBy("name").count()

category_film_set.withColumnRenamed("count", "Films Num").withColumnRenamed("name", "Category Name").sort(
    category_film_set["count"].desc()).show()

+-------------+---------+
|Category Name|Films Num|
+-------------+---------+
|       Sports|       74|
|      Foreign|       73|
|       Family|       69|
|  Documentary|       68|
|    Animation|       66|
|       Action|       64|
|          New|       63|
|        Drama|       62|
|        Games|       61|
|       Sci-Fi|       61|
|     Children|       60|
|       Comedy|       58|
|       Travel|       57|
|     Classics|       57|
|       Horror|       56|
|        Music|       51|
+-------------+---------+



2. Вивести 10 акторів, чиї фільми брали на прокат найбільше. Результат відсортувати за спаданням.

In [113]:
film_actors_set = rental_df.join(inventory_df, on=rental_df.inventory_id == inventory_df.inventory_id,
                                 how="inner").join(
    film_df, on=inventory_df.film_id == film_df.film_id, how="inner").join(
    film_actor_df, on=film_df.film_id == film_actor_df.film_id, how="inner").join(
    actor_df, on=film_actor_df.actor_id == actor_df.actor_id, how="inner").select(
    concat_ws(" ", actor_df.first_name, actor_df.last_name).alias("FullName"), rental_df.rental_id)

film_actors_set.select("FullName").groupBy("FullName").count().withColumnRenamed("count", "Rentals Num").sort(
    col("count").desc()).limit(10).show()


+------------------+-----------+
|          FullName|Rentals Num|
+------------------+-----------+
|       SUSAN DAVIS|        825|
|    GINA DEGENERES|        753|
|    MATTHEW CARREY|        678|
|       MARY KEITEL|        674|
|ANGELA WITHERSPOON|        654|
|       WALTER TORN|        640|
|       HENRY BERRY|        612|
|       JAYNE NOLTE|        611|
|        VAL BOLGER|        605|
|     SANDRA KILMER|        604|
+------------------+-----------+



3. Вивести категорія фільмів, на яку було витрачено найбільше грошей в прокаті

In [122]:
film_rental_set = rental_df.join(inventory_df, on=rental_df.inventory_id == inventory_df.inventory_id,
                                 how="inner").join(
    film_df, on=inventory_df.film_id == film_df.film_id, how="inner").join(
    film_category_df, on=film_df.film_id == film_category_df.film_id, how="inner").join(
    category_df, on=film_category_df.category_id == category_df.category_id, how="inner").join(
    payment_df, on=rental_df.rental_id == payment_df.rental_id, how="inner").select(
    category_df.name.alias("Category Name"), payment_df.amount)

film_rental_set.select(["Category Name", "amount"]).groupBy("Category Name").sum("amount").withColumnRenamed(
    "sum(amount)", "Films Rental Sum").sort(col("sum(amount)").desc()).limit(1).show()



+-------------+-----------------+
|Category Name| Films Rental Sum|
+-------------+-----------------+
|       Sports|5314.209999999848|
+-------------+-----------------+



4. Вивести назви фільмів, яких не має в inventory. Запит має бути без оператора IN

In [126]:
film_not_at_inventory_set = film_df.join(inventory_df, on=film_df.film_id == inventory_df.film_id, how="leftanti")
film_not_at_inventory_set.select(["title"]).show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5. Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [149]:
category_actors_set = film_df.join(film_actor_df, on=film_df.film_id == film_actor_df.film_id, how="inner").join(
    actor_df, on=film_actor_df.actor_id == actor_df.actor_id, how="inner").join(
    film_category_df, on=film_df.film_id == film_category_df.film_id, how="inner").join(
    category_df, on=film_category_df.category_id == category_df.category_id, how="inner").select(
    concat_ws(" ", actor_df.first_name, actor_df.last_name).alias("Full Name"), film_df.film_id).filter(
    category_df.name == "Children").groupBy("Full Name").count()

category_actors_set.select(["Full Name"]).sort(col("count").desc()).limit(3).show()



+------------+
|   Full Name|
+------------+
|HELEN VOIGHT|
| SUSAN DAVIS|
|  MARY TANDY|
+------------+



6. Вивести міста з кількістю активних та неактивних клієнтів (в активних customer.active = 1). Результат відсортувати за кількістю неактивних клієнтів за спаданням.

In [182]:
# Put to the data set active cities with client number
city_active_client_set = customer_df.join(address_df, on=customer_df.address_id == address_df.address_id,
                                          how="right").join(
    city_df, on=address_df.city_id == city_df.city_id, how="right").select(
    city_df.city.alias("city_active_client"), customer_df.active).filter(customer_df.active == 1).groupBy(
    "city_active_client").count().sort(
    col("count").desc())

# Put to the _another_ data set *non* active cities with client number
city_non_active_client_set = customer_df.join(address_df, on=customer_df.address_id == address_df.address_id,
                                              how="right").join(
    city_df, on=address_df.city_id == city_df.city_id, how="right").select(
    city_df.city.alias("city_non_active_client"), customer_df.active).filter(customer_df.active == 0).groupBy(
    "city_non_active_client").count().sort(
    col("count").desc())

# Join datasets from above with city_df
city_all_client_set = city_df.join(city_active_client_set, on=city_df.city == city_active_client_set.city_active_client,
                                   how='left').join(city_non_active_client_set,
                                                    on=city_df.city == city_non_active_client_set.city_non_active_client,
                                                    how='left').select(
    city_df.city,
    city_active_client_set["count"].alias("Active Customer Num"),
    city_non_active_client_set["count"].alias("Non Active Customer Num")).sort(
    col("Non Active Customer Num").desc()).na.fill(value=0).show()


+------------------+-------------------+-----------------------+
|              city|Active Customer Num|Non Active Customer Num|
+------------------+-------------------+-----------------------+
|         Pingxiang|                  0|                      1|
|       Szkesfehrvr|                  0|                      1|
|  Charlotte Amalie|                  0|                      1|
|         Najafabad|                  0|                      1|
|           Wroclaw|                  0|                      1|
|            Ktahya|                  0|                      1|
|            Amroha|                  0|                      1|
|           Bat Yam|                  0|                      1|
|   Southend-on-Sea|                  0|                      1|
|            Kamyin|                  0|                      1|
|          Xiangfan|                  0|                      1|
|            Daxian|                  0|                      1|
|          Uluberia|     